In [2]:
%cd "../"

d:\Documents\GitHub\onnx


In [2]:
import onnx
import onnxruntime as ort
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import zoom

# Input and preprocessing
image = np.load('samples/0001_0170.npy')
img_size = 128
x, y = image.shape
if x != img_size and y != img_size:
    inputs = zoom(image, (img_size / x, img_size / y), order=0)
    
inputs = inputs[np.newaxis,np.newaxis,:,:]

# Inference
EP_list = ['CUDAExecutionProvider', 'CPUExecutionProvider']
ort_session = ort.InferenceSession("onnx_model_sim.onnx", providers=EP_list)


In [3]:
inp = {ort_session.get_inputs()[0].name: inputs}
out = ort_session.run(None, inp)

out = np.argmax(out[0], axis=1)


In [5]:
%%timeit
inp = {ort_session.get_inputs()[0].name: inputs}
out = ort_session.run(None, inp)
out = np.argmax(out[0], axis=1)


8.35 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
import torch
import torch.nn as nn
import onnx
import onnxruntime as ort
import numpy as np
from time import time
from networks.UNet_plusplus.UNet_plusplus import UNet_plusplus

# Intialize model and input
model = torch.load("model.pth")
model.eval()
model = model.cuda()
model = torch.compile(model, backend="eager")

input = torch.rand(3, 1, 128, 128).to(torch.float32).cuda()
with torch.inference_mode():
    output = model(input)

# Export model to ONNX

In [10]:
%%timeit
with torch.inference_mode():
    output = model(input)

21 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
import torch
import torch.nn as nn


class Test(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(32, 64)
    
    def forward(self, x):
        dim = x.shape
        x = x * dim[0] + dim[1]
        return self.layer(x)

model = Test().cuda()
input = torch.rand(3, 32).to(torch.float32).cuda()
output = model(input)

# Export model to ONNX
torch.onnx.export(model, input, "onnx_model.onnx",
                  input_names=["input"], output_names=["output"])